# 統計2009年冬季到2018年冬季臺灣各縣市的空氣汙染物質

In [ ]:
from ipywidgets import interact_manual
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import numpy as np

%matplotlib inline

從行政院[環境保護署環境資源資料庫](https://erdb.epa.gov.tw/DataRepository/EnvMonitor/AirQualityMonitorMonData.aspx?topic1=%u5927%u6c23&topic2=%u74b0%u5883%u53ca%u751f%u614b%u76e3%u6e2c&subject=%u7a7a%u6c23%u54c1%u8cea)下載2008年12月到2019年1月的全臺的"空氣品質監測月值"至本機後 用pandas_read_csv抓取資料~<br>
或是從敝人的[google drive](https://drive.google.com/open?id=17hwzkzdy9ad50QH1alnbiVsXLeHFnRtQ)下載也可以喔~<br>
(政府資料開放平台好像只有2019年1月的月均值QQ)

## 前置作業
###    抓檔案並作資料整理
把檔案的路徑輸入pandas.read_csv的函數中 注意Windows路徑中的反斜線(\)要改成斜線(/)喔~

In [ ]:
air = pd.read_csv("C:/Users/User/Machine Learning/Homework_pandas/Air_Pollution_2008_Dec-2019_Jan.csv")
#air = pd.read_csv("C:/Users/User/Downloads/Air_Pollution_2008_Dec-2019_Jan.csv")
new_column = []
for i in air.columns.values:
    new_column.append(i.replace("<br />", ""))
new_column[4:13] = ['二氧化硫SO_2 (ppb)','一氧化碳CO (ppm)','二氧化碳CO_2 (ppm)','臭氧O_3 (ppb)','懸浮微粒PM10 (μg/m^3)','細懸浮微粒PM2.5 (μg/m^3)','氮氧化物NO_x (ppb)','一氧化氮NO (ppb)','二氧化氮NO_2 (ppb)']
A = zip(air.columns.values,new_column)
air.rename(columns=dict(A), inplace=True)
air = air.iloc[:-2,:].copy()
del new_column

### 偷看一下

In [ ]:
air.head()

### 將資料分成各年的DataFrame,並做成一個list: air_year

In [ ]:
air_year = []
y = 2008
m = 12
ym = []
ym.append(f'{y}/{m}')
year = []
for i in air['監測月份'].values:
    if i in ym:
        year.append(True)
    else:
        year.append(False) 
air_year.append(air[year])

for y in range(2009,2019):
    ym = []
    for m in range(0,13):
        if m<10:
            ym.append(f'{y}/0{m}')
        else:
            ym.append(f'{y}/{m}')
    year = []
    for i in air['監測月份'].values:
        if i in ym:
            year.append(True)
        else:
            year.append(False) 
    air_year.append(air[year])

y = 2019
m = 1
ym = []
ym.append(f'{y}/0{m}')
year = []
for i in air['監測月份'].values:
    if i in ym:
        year.append(True)
    else:
        year.append(False) 
air_year.append(air[year])

del ym; del year

### 檢查有沒有資料遺漏

In [ ]:
index = 0
for i in air_year:
    index += len(i.index)
#    print(f'{len(i.index)}', end=", ")
if index==len(air.index):
    print('沒缺資料喔~')
else:
    print('漏資料囉QQ')

### 做一些方便找資料的list (compounds, county, factors)及dictionaries (year_list, county_list)

In [ ]:
county=[]
for i in air['縣市'].values:
    if i not in county:
        county.append(i)

In [ ]:
year_list = dict(zip(np.arange(2008,2020),np.arange(0,12)))
county_list = dict(zip(county,np.arange(len(county))))
compounds = air.columns.values[4:13]
factors = air.columns[22:]

## 計算全國空汙物質的季度平均
1. 春季3-5月;夏季6-8月;秋季9-11月;冬季12-2月<br>
因為冬季會有跨年的問題 所以把上一年的12月歸類在下一年的冬季(所以資料才從2008/12開始)<br>
2. 計算各汙染物質平均時 會把全國各有效測站加總(但沒有加權) 再除以總共的有效測站數目 如果該測站沒有該汙染物質的觀測量 則當作無效的測站


In [ ]:
AP_compounds_season_country = []
for item in compounds:
    AP_season_country = np.zeros((len(year_list)-2)*4+1) #data is only available from 2008/12(winter) to 2019/1(winter)
    season = []
    count = 0
    Start = 2008
    End = 2019
    YEAR = Start

    while YEAR in range(Start,End):
        #winter
        month=12
        weight = 0
        temp = air_year[year_list[YEAR]][air_year[year_list[YEAR]]['監測月份'] == f'{YEAR}'+'/'+f'{(month)}']
        for i in temp[item].values:
            if i!='-':
                AP_season_country[count] += float(i)
                weight += 1
            
        YEAR = YEAR +1 # go to next year

        for month in range(1,3):
            temp = air_year[year_list[YEAR]][air_year[year_list[YEAR]]['監測月份'] == f'{YEAR}'+'/0'+f'{month}']
            for i in temp[item].values:
                if i!='-':
                    AP_season_country[count] += float(i)
                    weight += 1
        if weight!=0:
            AP_season_country[count] /= weight
            season.append(f'{YEAR}'+'冬')
        else:
            print(f'No available data in {YEAR} winter!\n')
        count +=1
    
        # insert this after the season in which data ends
        if YEAR>=End:
            break
    
        #spring
        weight =0
        for month in range(3,6):
            temp = air_year[year_list[YEAR]][air_year[year_list[YEAR]]['監測月份'] == f'{YEAR}'+'/0'+f'{month}']
            for i in temp[item].values:
                if i!='-':
                    AP_season_country[count] += float(i)
                    weight += 1
        if weight!=0:
            AP_season_country[count] /= weight
            season.append(f'{YEAR}'+'春')
        else:
            print(f'No available data in {YEAR} spring!\n')
        count +=1

        #summer
        weight =0
        for month in range(6,9):
            temp = air_year[year_list[YEAR]][air_year[year_list[YEAR]]['監測月份'] == f'{YEAR}'+'/0'+f'{month}']
            for i in temp[item].values:
                if i!='-':
                    AP_season_country[count] += float(i)
                    weight += 1
        if weight!=0:
            AP_season_country[count] /= weight
            season.append(f'{YEAR}'+'夏')
        else:
            print(f'No available data in {YEAR} summer!\n')
        count +=1

        #fall
        weight =0
        month=9
        temp = air_year[year_list[YEAR]][air_year[year_list[YEAR]]['監測月份'] == f'{YEAR}'+'/0'+f'{month}']
        for i in temp[item].values:
            if i!='-':
                AP_season_country[count] += float(i)
                weight += 1
        for month in range(10,12):
            temp = air_year[year_list[YEAR]][air_year[year_list[YEAR]]['監測月份'] == f'{YEAR}'+'/'+f'{month}']            
            for i in temp[item].values:
                if i!='-':
                    AP_season_country[count] += float(i)
                    weight += 1
        if weight!=0:
            AP_season_country[count] /= weight
            season.append(f'{YEAR}'+'秋')
        else:
            print(f'No available data in {YEAR} fall!\n')
        
        count += 1
        
    AP_compounds_season_country.append(AP_season_country)

whole_country = pd.DataFrame(AP_compounds_season_country,index=compounds, columns=season)
whole_country = whole_country.T # .T means transpose the data
whole_country.head()

## 計算各縣市平均
###    把各月份的值歸類到所屬的季度 並依據縣市做成各自的 DataFrame 方便以後呼叫

In [ ]:
def AIR_POLLUTION_L(location):
    AP_compounds_season_location = []
    for item in compounds:
        AP_season_location = np.zeros((len(year_list)-2)*4+1) #data is only available from 2008/12(winter) to 2019/1(winter)
#        season = []
        count = 0
        Start = 2008
        End = 2019
        YEAR = Start

        while YEAR in range(Start,End):
            #winter
            month=12
            weight = 0
            AP_year_location = air_year[year_list[YEAR]][air_year[year_list[YEAR]]['縣市']==location]
            temp = AP_year_location[AP_year_location['監測月份'] == f'{YEAR}'+'/'+f'{(month)}']
            for i in temp[item].values:
                if i!='-':
                    AP_season_location[count] += float(i)
                    weight += 1
            
            YEAR = YEAR +1 # go to next year

            AP_year_location = air_year[year_list[YEAR]][air_year[year_list[YEAR]]['縣市']==location]
            for month in range(1,3):
                temp = AP_year_location[AP_year_location['監測月份'] == f'{YEAR}'+'/0'+f'{month}']
                for i in temp[item].values:
                    if i!='-':
                        AP_season_location[count] += float(i)
                        weight += 1
            if weight!=0:
                AP_season_location[count] /= weight
#                season.append(f'{YEAR}'+'冬')
            else:
                print(f'\t\tNo available {item} data in {YEAR} winter!\n')
            count +=1
    
             #insert this after the season in which data ends
            if YEAR>=End:
                break
    
            #spring
            weight =0
            for month in range(3,6):
                temp = AP_year_location[AP_year_location['監測月份'] == f'{YEAR}'+'/0'+f'{month}']
                for i in temp[item].values:
                    if i!='-':
                        AP_season_location[count] += float(i)
                        weight += 1
            if weight!=0:
                AP_season_location[count] /= weight
#                season.append(f'{YEAR}'+'春')
            else:
                print(f'\t\tNo available {item} data in {YEAR} spring!\n')
            count +=1

            #summer
            weight =0
            for month in range(6,9):
                temp = AP_year_location[AP_year_location['監測月份'] == f'{YEAR}'+'/0'+f'{month}']
                for i in temp[item].values:
                    if i!='-':
                        AP_season_location[count] += float(i)
                        weight += 1
            if weight!=0:
                AP_season_location[count] /= weight
#                season.append(f'{YEAR}'+'夏')
            else:
                print(f'\t\tNo available {item} data in {YEAR} summer!\n')
            count +=1

            #fall
            weight =0
            month=9
            temp = AP_year_location[AP_year_location['監測月份'] == f'{YEAR}'+'/0'+f'{month}']
            for i in temp[item].values:
                if i!='-':
                    AP_season_location[count] += float(i)
                    weight += 1
            for month in range(10,12):
                temp = AP_year_location[AP_year_location['監測月份'] == f'{YEAR}'+'/'+f'{month}']            
                for i in temp[item].values:
                    if i!='-':
                        AP_season_location[count] += float(i)
                        weight += 1
            if weight!=0:
                AP_season_location[count] /= weight
#                season.append(f'{YEAR}'+'秋')
            else:
                print(f'\t\tNo available {item} data in {YEAR} fall!\n')
            count += 1
        AP_compounds_season_location.append(AP_season_location)
    return (AP_compounds_season_location)

### 執行上面的函數(如果該季度沒有觀測值的話會跟你說喔 像CO<sub>2</sub>只有少數測站有觀測到 所以很多地點都沒資料...) 最後再把所有縣市加上全國的 DataFrame 合成一個 list: air_season

In [ ]:
air_season = []
print ('計算各縣市各年份季節平均空汙物質\n\n我是處理報表')
for i in county:
    print (f'\t現在處理{i}...')
    temp = pd.DataFrame(AIR_POLLUTION_L(i), index=compounds, columns= season)
    air_season.append(temp.T)
air_season.append(whole_country)
print('\t完成囉!!')

### 將結果作圖(可以你居住的地方或老家看一看嚇自己喔~)
先定義畫圖的函數

In [ ]:
def AP_LOCAL_ALL_SEASON(location, item):
    plt.rcdefaults()
    plt.style.use('ggplot')
    plt.figure(figsize=(30,15),dpi=200)
    plt.plot(air_season[county_list[location]].index, air_season[county_list[location]][item].values,color='blue',lw=5.0,marker='o',markersize=20.0,label=f'{location}')
    plt.plot(air_season[-1].index, air_season[-1][item].values,color='red',lw=5.0,marker='o',markersize=20.0,label='全國平均')
    plt.xticks(range(0,41),season, rotation=45,fontsize=20)
    plt.yticks(fontsize=20)
    plt.xlim(-1,41)
    plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei']
    plt.xlabel('年份-季節',fontsize=20)
    
    item = item.replace('_2', '$_{2}$')
    item = item.replace('_3', '$_{3}$')
    item = item.replace('_x', '$_{x}$')
    item = item.replace('μ', '$\mu$')
    item = item.replace('^3', '$^{3}$')
    
    plt.ylabel(r'{}'.format(item),fontsize=20)
    plt.title( r'{} 2009冬-2019冬 {} 趨勢圖'.format(location,item),fontsize=30)
    plt.legend(loc='best', prop={'size': 30}, facecolor='white') #prop adjust the legend fontsize
    plt.show()

In [ ]:
def AP_ALL_COUNTY_SEASON(time, item):
    AP_area_comparsion = []
    label = []
    for i in county:
        temp = air_season[county_list[i]].loc[time,item]
        if temp>0:
            AP_area_comparsion.append(temp)
            label.append(i)
    plt.rcdefaults()
    plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei']
    plt.style.use('seaborn-notebook')
    color = ['r','orange','yellow', 'limegreen','b','purple','brown','gold','silver','lime','cyan','magenta','red','darkorange','greenyellow','chocolate','aqua','m','lawngreen','orangered','deepskyblue','indigo']
    fig = plt.figure(figsize=(5,15), dpi=200)
    
    ITEM = item # for calling the whole country average value
    item = item.replace('_2', '$_{2}$')
    item = item.replace('_3', '$_{3}$')
    item = item.replace('_x', '$_{x}$')
    item = item.replace('μ', '$\mu$')
    item = item.replace('^3', '$^{3}$')
    
    ax0 = plt.subplot2grid((2,1),(0,0))
    ax0 = plt.pie(AP_area_comparsion, labels=label, colors = color, autopct='%1.2f%%', shadow=False, startangle=0,radius=2.0)
    ax0 = plt.title( r'{} 各縣市 {} 百分比'.format(time,item),fontsize=30, y = 1.4) # y adjust the poisition of the title

    temp = pd.DataFrame({'county':label,item:AP_area_comparsion}).sort_values(by=[item],ascending=False)
    ax1 = plt.subplot2grid((2,1),(1,0))
    # use iloc because the index is not changed after sorting, print temp then you can see!
    ax1 = plt.bar(temp.iloc[0:5,0],temp.iloc[0:5,1],color=color)
    ax1 = plt.ylabel(r'{}'.format(item),fontsize=20)
    ax1 = plt.axhline(air_season[-1].loc[time,ITEM],linestyle=':',color='black',)
    ax1 = plt.text(4.75,air_season[-1].loc[time,ITEM]*0.975,'全臺平均值')
    ax1 = plt.title( r'{} 各縣市 {} 排名'.format(time,item),fontsize=30, y=-0.15) # y adjust the poisition of the title
    plt.show()

In [ ]:
def AP_FACTOR(reason, item):
    x = []
    temp = air.loc[:,reason][(air.loc[:,reason]!='-')&(air.loc[:,item]!='-')]
    for i in temp:
        x.append(float(i))
    x = np.array(x)
    y = []
    temp = air.loc[:,item][(air.loc[:,reason]!='-')&(air.loc[:,item]!='-')]
    for i in temp:
        y.append(float(i))
    y = np.array(y)
    
    if len(x)!=0 and len(y)!=0: 
        plt.rcdefaults()
        plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei']
        plt.style.use('seaborn-notebook')
        plt.figure(figsize=(10,10), dpi=200)
        plt.grid(alpha=0.5,ls=':')
    
        item = item.replace('_2', '$_{2}$')
        item = item.replace('_3', '$_{3}$')
        item = item.replace('_x', '$_{x}$')
        item = item.replace('μ', '$\mu$')
        item = item.replace('^3', '$^{3}$')
        reason = reason.replace('μ', '$\mu$')
                            
        plt.scatter(x,y,s=60.0)
        plt.xlabel(r'{}'.format(reason),fontsize=20)
        plt.ylabel(r'{}'.format(item),fontsize=20)
        plt.title(r'{} 與 {} 的相關性'.format(item,reason), fontsize=30, y=1.05)
        #x_max = x.max()
        #x_min = x.min()
        #y_max = y.max()
        #y_min = y.min()
        #sx = x_max-x_min
        #sy = y_max-y_min
        #plt.xlim(x_min-0.05*sx,x_max+0.05*sx)
        #plt.ylim(y_min-0.05*sx,y_max+0.05*sx)
    
        #plt.xticks(np.arange(x_min,x_max+0.1*sx,0.1*sx))
        #plt.yticks(np.arange(y_min,y_max+0.1*sy,0.1*sy))
        
    else:
        print('沒資料點可以畫圖喔QQ')

### 指定縣市及汙染物質對季度-年份作圖
請選擇地點及想查詢的汙染物質

In [ ]:
interact_manual(AP_LOCAL_ALL_SEASON,location=county, item=compounds);

1. 可以觀察到各空氣汙染物質都是在冬天比較高(蠻意外CO<sub>2</sub>也是，以為大家都在吹冷氣所以火力發電量比較大XD) 可能是因為空氣比較乾燥的關係嗎~ 果然冬天躲在家裡最棒了!<br>
2. 大致上:<br>
二氧化硫SO<sub>2</sub>,一氧化碳CO,一氧化氮NO,二氧化氮NO<sub>2</sub>,氮氧化物NO<sub>x</sub>,PM10, PM2.5<br> 逐年減少
臭氧O<sub>3</sub>持平<br>
二氧化碳CO<sub>2</sub>逐年增加


### 指定年份及汙染物質對各縣市作圓餅圖 並將排名前五的縣市作長條圖(黑虛線是全臺平均值)
請選擇 年份-季節 以及 想查詢的汙染物質

In [ ]:
interact_manual(AP_ALL_COUNTY_SEASON, time=season, item=compounds);

1. 果然高雄冬天PM2.5很多 QQ
2. 台東花蓮空氣品質真的讚 OWOb
3. 新北跟台北好多一氧化氮NO(這是不是有毒性啊)喔~ 不知道為什麼@W@a
4. 離島冬天的空汙也頗嚴重 (>口<)

### 用散佈圖觀察汙染物質是否跟其他大氣條件有關係
可能影響汙染物質的大氣條件可從 factors 這個 list 查詢喔

In [ ]:
interact_manual(AP_FACTOR,reason=factors,item=compounds);

1. 一氧化氮NO,二氧化氮NO<sub>2</sub>,氮氧化物NO<sub>x</sub>,二氧化硫SO<sub>2</sub>,一氧化碳CO,二氧化碳CO<sub>2</sub>, PM10, PM2.5好像與風速呈反比 臭氧O<sub>3</sub>則有正比的趨勢 該不會是被吹過來的 OWOa